In [1]:
import gym

In [2]:
env = gym.make("HandManipulateBlock-v0")

In [3]:
print("Action Space Range:")
print(env.action_space.high)
print(env.action_space.low)
print("\nAction Space channels: ",len(env.action_space.high))

Action Space Range:
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1.]

Action Space channels:  20


These environments are based on the **Shadow Dexterous** Hand,5 which is an anthropomorphic robotic
hand with **24 degrees of freedom**. Of those 24 joints, 20 can be can be controlled independently
whereas the remaining ones are coupled joints.

In all hand tasks, rewards are sparse and binary:
The agent obtains a reward of 0 if the goal has been achieved (within some task-specific tolerance) and −1 otherwise.
**Actions are 20-dimensional**: We use absolute position control for all non-coupled joints of the hand. We apply the same action in 20 subsequent simulator steps (with ∆t = 0.002 each) before returning control to the agent, i.e. the agent’s action frequency is f = 25 Hz.

Observations include the 24 positions and velocities of the robot’s joints. In case of an object that is being manipulated, we also include its Cartesian position and rotation represented by a quaternion (hence 7-dimensional) as well as its linear and angular velocities. In the reaching task, we include the Cartesian position of all 5 fingertips.

### Reaching (*HandReach-v0*):
A simple task in which the goal is 15-dimensional and contains the target
Cartesian position of each fingertip of the hand. Similarly to the FetchReach task, this task is
relatively easy to learn. A goal is considered achieved if the mean distance between fingertips and
their desired position is less than 1 cm.

In [4]:
import serial
import numpy as np
import os
import gc
import time

In [5]:
class config():
    
    #SERIAL SETTINGS
    SERIAL_PORT = "COM5"
    SERIAL_BAUDRATE = 9600
    
    # TESTED 180 MEASURES/SECOND
    MEASURMENTS_PER_SECOND = 180 #ACTUAL RANGE 180-186
    MEASUREMENT_TIME_WINDOW = 0.1 #seconds
    NOISE_FILTER = 120 #SIGNAL -> SIGNAL - NOISE_FILTER
    
    #NORMALIZATION
    SIGNAL_RANGE = (0, 500 - NOISE_FILTER)#SERIAL FULL RANGE (0,1023) BUT MAXIMUM MUSCLE FLEX REACHES 800-ish
    ACTION_SPACE_RANGE = (-1,1)

In [6]:
ser = serial.Serial(config.SERIAL_PORT, baudrate=config.SERIAL_BAUDRATE, timeout=1)

if not ser.is_open:
    ser.open()
    
ser.reset_input_buffer()

In [7]:
def getOptimalMeanWindow(measures_per_s, time_window_in_s=0.1):
    return measures_per_s * time_window_in_s
mean_window = getOptimalMeanWindow(config.MEASURMENTS_PER_SECOND, config.MEASUREMENT_TIME_WINDOW)
print(mean_window)

18.0


In [8]:
def getEMGsignal(ser, mean_window=mean_window, noise_filter=config.NOISE_FILTER, verbose=False):
    if not ser.is_open:
        ser.open()
    ser.reset_input_buffer()
    out = []
    while True:
        try:
            try:
                a0_value = int(ser.readline().decode().strip())
                if verbose:
                    print(a0_value)
                out.append(a0_value - noise_filter)
                if len(out) == mean_window:
                    return sum(out) / len(out)
            except KeyboardInterrupt:
                print("KeyboardInterrupt")
                ser.close()
                break
            except Exception as e:
                #print(e)
                continue
        except Exception as e:
            print(e)
            ser.close()
            raise e
        except KeyboardInterrupt:
            print("KeyboardInterrupt")
            ser.close()
            break
    ser.close()

def norm_signal(signal, signal_range=config.SIGNAL_RANGE, norm_range=config.ACTION_SPACE_RANGE):
    return ( (signal - signal_range[0]) / (signal_range[1] - signal_range[0]) * (norm_range[1] - norm_range[0]) ) + norm_range[0]

In [9]:
print(norm_signal(0),"\n",norm_signal(380),"\n",norm_signal(300))

-1.0 
 1.0 
 0.5789473684210527


In [10]:
while True:
    action = norm_signal(getEMGsignal(ser))
    print(action)

-0.4766081871345029
-0.39532163742690063
-0.2973684210526316
-0.24941520467836253
-0.3263157894736842
-0.37339181286549705
-0.34444444444444444
-0.32748538011695905
-0.2976608187134502
-0.38421052631578945
-0.4476608187134503
-0.33216374269005855
-0.3769005847953216
-0.362280701754386
-0.3991228070175439
-0.4099415204678363
-0.3728070175438596
-0.3991228070175439
-0.4122807017543859
-0.46725146198830403
-0.5043859649122806
-0.48099415204678364
-0.502046783625731
-0.4631578947368421
-0.43625730994152045
-0.46374269005847957
-0.4432748538011696
-0.39649122807017545
-0.2728070175438597
-0.34269005847953216
-0.4245614035087719
KeyboardInterrupt



KeyboardInterrupt



In [10]:

for i_episode in range(20):
    try:
        obs = env.reset()
        for t in range(100):
            try:
                env.render()
                #print(obs)
                action = np.zeros(len(env.action_space.high))
                #Clipping is automatic
                action[1] = norm_signal(getEMGsignal(ser))
                obs, reward, done, info = env.step(action)
                if done:
                    print("Episode finished after {} timesteps".format(t+1))
                    break
            except KeyboardInterrupt:
                print("Interrupted")
                env.close()
    except KeyboardInterrupt:
        print("Interrupted")
        env.close()
        break
        
env.close()

Creating window glfw
Episode finished after 100 timesteps
Episode finished after 100 timesteps
Episode finished after 100 timesteps
Episode finished after 100 timesteps
Episode finished after 100 timesteps
Episode finished after 100 timesteps
Episode finished after 100 timesteps
Episode finished after 100 timesteps
Episode finished after 100 timesteps
Episode finished after 100 timesteps
Episode finished after 100 timesteps
Episode finished after 100 timesteps
Episode finished after 100 timesteps
Episode finished after 100 timesteps
Episode finished after 100 timesteps
Episode finished after 100 timesteps
Episode finished after 100 timesteps
Episode finished after 100 timesteps
Episode finished after 100 timesteps
Episode finished after 100 timesteps
